In [1]:
from typing import List
import pandas as pd
import json
import os

def compute_validation_effectiveness(baseline_file: str, model_names: List, result_dir: str, index_name: str):

    df_baseline = pd.read_csv(baseline_file)
    
    results = []

    for model_name in model_names:
        
        results_file = f"{result_dir}/all_dependencies_{index_name}_{model_name}.json"

        with open(results_file, "r", encoding="utf-8") as src:
            data = json.load(src)

        true_positives = []
        true_negatives = []
        false_positives = []
        false_negatives = []
        accuracy_count = []

        for entry, (index, row) in zip(data, df_baseline.iterrows()):

            rating = row["final_rating"]
            try:
                if "responses" in entry:
                    response = entry["responses"][0]
                else:
                    response = entry["response"]
                response_dict = json.loads(response)
                isDependency = response_dict["isDependency"]
            except json.JSONDecodeError as error:
                #print(index)
                #print(response_dict)
                #print(type(response_dict))
                try:
                    isDependency = response_dict["isDependency"]
                except KeyError as error:
                    print(error)
                    continue
            except KeyError as error:
                print(error)
                continue
            
            if str(rating) == "Borderline":
                if isDependency:
                    accuracy_count.append(1)
                    true_positives.append(1)
                if not isDependency:
                    accuracy_count.append(1)
                    true_negatives.append(1)

            # TP: The LLM validates a dependency as correct and the dependency is correct
            if isDependency and str(rating).lower() == "true":
                accuracy_count.append(1)
                true_positives.append(1)
                
            # FP: The LLM validates a dependency as correct, but the dependency is actually incorrect
            if isDependency and str(rating).lower() == "false":
                accuracy_count.append(0)
                false_positives.append(1)

            # TN: The LLM validates a dependency as incorrect and the dependency is incorrect
            if not isDependency and str(rating).lower() == "false":
                accuracy_count.append(1)
                true_negatives.append(1)

            # FN: The LLM validates a dependency as incorrect, but the dependency is actually correct
            if not isDependency and  str(rating).lower() == "true":
                accuracy_count.append(0)
                false_negatives.append(1)

        
        tp = sum(true_positives)
        fp = sum(false_positives)
        fn = sum(false_negatives)
        tn = sum(true_negatives)
        accuracy = sum(accuracy_count)/len(accuracy_count)


        print("Model: ", model_name)
        print("File: ", results_file)
        print("TP", tp)
        print("FP", fp)
        print("TN", tn)
        print("FN", fn)
        print("Sum correct response format: ", sum([tp, fp, tn, fn]))

        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1_score = 2 * (precision * recall) / (precision + recall)

        print("Accuracy", accuracy)
        print("Precision", precision)
        print("Recall", recall)
        print("F1 Score: ", f1_score)

        results.append({
            "model_name": model_name,
            "file": results_file,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1-score": f1_score
        })
    

    df_results = pd.DataFrame(results)
    df_results.to_csv(f"{result_dir}/validation_effectiveness.csv", index=False)


compute_validation_effectiveness(
    baseline_file="../data/results/all_dependencies.csv",
    model_names=["gpt-3.5-turbo-0125", "gpt-4o-2024-05-13", "llama3:70b", "llama3:8b"],
    result_dir="../data/results/config2",
    index_name="all"
)

Model:  gpt-3.5-turbo-0125
File:  ../data/results/config2/all_dependencies_all_gpt-3.5-turbo-0125.json
TP 160
FP 137
TN 160
FN 43
Sum correct response format:  500
Accuracy 0.64
Precision 0.5387205387205387
Recall 0.7881773399014779
F1 Score:  0.64
Model:  gpt-4o-2024-05-13
File:  ../data/results/config2/all_dependencies_all_gpt-4o-2024-05-13.json
TP 117
FP 30
TN 267
FN 86
Sum correct response format:  500
Accuracy 0.768
Precision 0.7959183673469388
Recall 0.5763546798029556
F1 Score:  0.6685714285714285
Model:  llama3:70b
File:  ../data/results/config2/all_dependencies_all_llama3:70b.json
TP 147
FP 113
TN 184
FN 56
Sum correct response format:  500
Accuracy 0.662
Precision 0.5653846153846154
Recall 0.7241379310344828
F1 Score:  0.6349892008639308
'isDependency'
'isDependency'
'isDependency'
'isDependency'
Model:  llama3:8b
File:  ../data/results/config2/all_dependencies_all_llama3:8b.json
TP 179
FP 243
TN 50
FN 24
Sum correct response format:  496
Accuracy 0.46169354838709675
Precisio

In [2]:
import pandas as pd

file_path = "../data/results/config2/validation_effectiveness.csv"

df = pd.read_csv(file_path)

df

,model_name,file,accuracy,precision,recall,f1-score
0,gpt-3.5-turbo-0125,../data/results/config2/all_dependencies_all_g...,0.640000,0.538721,0.788177,0.640000
1,gpt-4o-2024-05-13,../data/results/config2/all_dependencies_all_g...,0.768000,0.795918,0.576355,0.668571
2,llama3:70b,../data/results/config2/all_dependencies_all_l...,0.662000,0.565385,0.724138,0.634989
3,llama3:8b,../data/results/config2/all_dependencies_all_l...,0.461694,0.424171,0.881773,0.572800
